In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
#Basic Imports
import os,sys
os.chdir("..")

from tqdm import tqdm,trange
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay
import torch
import pandas

from datasets.ssl_dataset import SSL_Dataset
from datasets.data_utils import get_data_loader
from utils import get_model_checkpoints_and_timing_info
from utils import net_builder
from utils import clean_results_df

In [ ]:
#Path to the runs to load
csv_folder = "/home/gabrielemeoni/project/SSLRS/"
dataset="eurosat_ms"
folder = "/scratch/fixmatch_results/sandy_runs/nr_of_labels/"
sort_criterion = "net" # Accepted net, numlabels
seed_wanted = 2# Seed wanted (the others will be filtered)

## Initialize parameters

In [ ]:
folder=os.path.join(folder, dataset)
checkpoints, run_args, timing_info = get_model_checkpoints_and_timing_info(folder)
timing_info_dict = dict(zip(checkpoints, timing_info))
if os.name == 'nt':
       [print(_.split("\\")[1]) for _ in checkpoints];
else:
       [print(_.split("/")[1]) for _ in checkpoints];

## Run all models

In [ ]:
results=[]
for checkpoint, args in zip(checkpoints,run_args):
    print("------------ RUNNING ", checkpoint, " -----------------")
    args["batch_size"] = 256
    args["data_dir"] = "./data/"
    args["use_train_model"] = False
    args["load_path"] = checkpoint
    if args["seed"] == seed_wanted:
        results.append({"params" : args, "aumgent_time" : timing_info_dict[checkpoint][0], "run_time" : timing_info_dict[checkpoint][1], "training_time" : timing_info_dict[checkpoint][2]})

In [ ]:
import pandas as pd
big_df = pd.DataFrame()
pd.set_option('display.max_columns', None)
for result in results:
    result_dict=result["params"].copy()
    result_dict.update({"aumgent_time" : result["aumgent_time"]})
    result_dict.update({"run_time" : result["run_time"]} )
    result_dict.update({"training_time" : result["training_time"]})
    df = pd.DataFrame(result_dict, index=[0])
    df = df.set_index(["dataset"])
    df = df.drop(labels=["batch_size","data_dir", "opt", "use_train_model", "seed", "confidence", "lr"], axis=1)
    big_df = big_df.append(df)

big_df=big_df.sort_values(by=[sort_criterion], axis=0)
big_df.to_csv(os.path.join(csv_folder,"time_results_"+dataset+"_seed_"+str(seed_wanted)+".csv"))
print("File saved at: ", os.path.join(csv_folder,"time_results_"+dataset+"_seed_"+str(seed_wanted)+".csv"))